In [6]:
import keras
keras.__version__
from keras import models
from keras import layers


import numpy as np
import pandas as pd
import re 

In [14]:
#train = pd.read_csv('C:/Users/Kevin/Desktop/Machine Learning/Assignment_2/train.json')
train = pd.read_json('train.json')
test = pd.read_json('test.json')

#20 unique classes
train['cuisine'].nunique()

20

In [15]:


"""
Provides functions for removing unwanted noise from text.
"""

# Moving lists like this outside the function definition make the function code more readable and 
# less verbose.
character_exceptions_to_remove = ["\xae", "\xe8vre", "\u2122"]
phrase_exceptions = [("7 up", "sevenup"), ("sun dried", "sundried"), ("bone less", "boneless"), 
                     ("skin less", "skinless")]
map_plural_to_singular = [("steaks", "steak"), ("loins", "loin"), ("inches", "inch"), ("centimeters", "centimeter"),
                          ("ounces", "ounce"), ("liters", "liter"), ("mililiters", "mililiter"), ("grams", "gram"),
                          ("cups", "cup"), ("gallons", "gallon"), ("quarts", "quart"), ("lbs", "lb"),
                          ("pounds", "pound"), ("tablespoons", "tablespoon"), ("teaspoons", "teaspoon"), 
                          ("pints", "pint"), ("fluid ounces", "fluid ounce"), ("onions", "onion"), 
                          ("cloves", "clove"), ("bulbs", "bulb"), ("peppers", "pepper"), ("breasts", "breast"),
                          ("eggs", "egg"), ("carrots", "carrot"), ("mushrooms", "mushroom"),
                          ("tortillas", "tortilla"), ("sausages", "sausage"), ("wedges", "wedge"), 
                          ("tomatoes", "tomato"), ("thighs", "thigh"), ("chilies", "chili"), ("potatoes", "potato"), 
                          ("peppercorns", "peppercorn"), ("spices", "spice"), ("chiles", "chile"), ("apples", "apple"),
                          ("legs", "leg"), ("doughs", "dough"), ("drumsticks", "drumstick")]
brandnames_to_remove = ["alexia", "breakstones", "kraft", "bertolli classico", "bertolli", "best foods", 
                        "betty crocker", "bisquick", "bob evans", "breyers", "curry guy", "camellia", "campbells", 
                        "country crock", "crisco", "crystal farms", "delallo", "diamond crystal", "domino", 
                        "doritos", "earth balance", "egglands best", "foster farms", "franks", "gold medal", 
                        "goya", "green giant steamers niblets", "green giant", "heinz", "hellmanns", "herdez", 
                        "hidden valley", "honeysuckle white", "jacksonville",  "jimmy dean", "johnsonville", 
                        "knorr", "krudsen", "kikkoman", "lipton", "land o lakes", "mazola", "lea and perrins", 
                        "mccormick", "meyer", "mission", "old el paso", "old bay", "pam", "pepperidge farm", 
                        "oscar mayer", "pace", "pillsbury", "progresso", "pure wesson", "pompeian", "san marzano", 
                        "sargento", "soy vay", "taco bell", "yoplait", "spice islands", "stonefire", "success", 
                        "swanson", "truvía", "uncle bens", "wish bone", "zatarains", "morton", "jameson", "tapatio", 
                        "mountain high", "philadelphia", "king arthur", "roma"]
keywords_to_remove = ["lowfat", "light", "shredded", "sliced", "all purpose", "all natural", "natural", "original", 
                      "gourmet", "traditional", "boneless", "skinless", "fresh", "nonfat", "pitted", "quick cooking", 
                      "unbleached", "part skim", "skim", "quickcooking", "oven ready", "homemade", "instant", "small", 
                      "extra large", "large", "chopped", "grated", "cooked", "stone ground", "freshly ground", 
                      "ground", "pure", "peeled", "deveined", "organic", "cracked", "granulated", "inch thick", 
                      "extra firm", "crushed", "flakes", "self rising", "diced", "crumbles", "crumbled", 
                      "whole wheat", "whole grain", "baby", "medium", "plain", "of", "thick cut", "cubed", "coarse", 
                      "free range", "seasoned", "canned", "multipurpose", "vegan", "thawed", "squeezed", 
                      "vegetarian", "fine", "zesty", "halves", "firmly packed", "drain", "drained", "washed"]
measurements_to_remove = ["in", "inch", "cm", "centimeter", "oz", "ounce", "l", "liter", "ml", "mililiter", "g", 
                          "gram", "cup", "gallon", "quart", "lb", "pound", "tbsp", "tablespoon", "tsp", "teaspoon", 
                          "pint", "fl oz", "fluid ounce"]
phrases_to_remove = measurements_to_remove + keywords_to_remove + brandnames_to_remove
phrases_to_map = [
    (("green onion", "red onion", "purple onion", "yellow onion", "yel onion"), "onion"),
    (("collard green leaves", "collards", "collard leaves"), "collard greens"),
    ("black pepper", "pepper"),
    ("yel chives", "chives"),
    ("spinach leaves", "spinach"),
    ("tea leaves", "tea"),
    ("chile", "chili"),
    (("garlic clove", "garlic bulb"), "garlic"),
    ("uncooked", "raw"),
    (("red chili pepper", "hot chili pepper", "red hot chili pepper"), "chili pepper"),
    (("baking potato", "baked potato"), "baked potato"),
    (("sea salt", "kosher salt", "table salt", "white salt"), "salt"),
    ("scotch whiskey", "scotch"),
    (("i cant believe its not butter spread", "i cant believe its not butter"), "butter"),
    (("extra virgin olive oil", "virgin olive oil"), "olive oil"),
    (("white bread", "wheat bread", "grain bread"), "bread"),
    (("white sugar", "yel sugar"), "sugar"),
    ("confectioners sugar", "powdered sugar")
]

# When executing multiple regex parses, it's most efficient to compile the expression ahead of time.
punctuation_to_replace_with_space = re.compile(r"[-,]")
percentage_less_to_remove = re.compile(r"[\d+]% less [A-z]*")
percentage_reduced_to_remove = re.compile(r"[\d+]% reduced [A-z]*")
symbols_to_remove = re.compile(r"[!\\/%.'®™]")
digits_to_remove = re.compile(r"\d+")
symbols_to_replace_with_and = re.compile(r"[&+]")
parentheses_content_to_remove = re.compile(r"\([^)]*\)")
no_blank_added_to_remove = re.compile(r"no [A-z]* added")
reduced_and_following_word_to_remove = re.compile(r"reduced [A-z]*")
low_and_following_word_to_remove = re.compile(r"low [A-z]*")
less_and_following_word_to_remove = re.compile(r"less [A-z]*")
free_and_previous_word_to_remove = re.compile(r"[A-z]* free")
multiple_spaces_to_trim = re.compile(r" +")


def clean_ingredient(ingredient):
    """
    Removes unnecessary noise from the inputted text.
    """
    
    # Convert to lowercase.
    ingredient = ingredient.lower()
    
    # Replace hyphens and commas with spaces.
    ingredient = punctuation_to_replace_with_space.sub(" ", ingredient)
    
    # Map certain exceptions that we don't want whiped out by later cleaning processes.
    for character_exception in character_exceptions_to_remove:
        ingredient = re.sub(character_exception, "", ingredient)
    for phrase_exception, replacement in phrase_exceptions:
        ingredient = re.sub(r"\b{}\b".format(phrase_exception), replacement, ingredient)
        
    # Remove "percentage less" instances (e.g., "40% less sodium") - do this before removing % and digits.
    ingredient = percentage_less_to_remove.sub("", ingredient)
    
    # Remove "percentage reduced" instances (e.g., "50% reduced fat") - do this before removing % and digits.
    ingredient = percentage_reduced_to_remove.sub("", ingredient)
    
    # Remove various unwanted symbols.
    ingredient = symbols_to_remove.sub("", ingredient)
    
    # Remove digits.
    ingredient = digits_to_remove.sub("", ingredient)
    
    # Replace common symbols with their word equivalent (so "Ben&Jerry's" == "Ben and Jerry's", etc.).
    ingredient = symbols_to_replace_with_and.sub(" and ", ingredient)
    
    # Remove anything between parentheses (this mainly includes volume measurements or 
    # unnecessary cooking instructions).
    ingredient = parentheses_content_to_remove.sub(" ", ingredient)
    
    # Remove all instances of 'no [sugar, sodium, fat, whatever] added'.
    ingredient = no_blank_added_to_remove.sub("", ingredient)
    
    # Remove any instance of 'reduced' and the word after it (e.g., 'reduced sodium').
    ingredient = reduced_and_following_word_to_remove.sub("", ingredient)
    
    # Remove any instance of 'low' and the word after it (e.g., 'low fat', 'low sodium').
    ingredient = low_and_following_word_to_remove.sub("", ingredient)
    
    # Remove any instance of 'less' and the word after it (e.g., 'less sodium').
    ingredient = less_and_following_word_to_remove.sub("", ingredient)
    
    # Remove and instance of 'free' and the word before it (e.g., 'sugar free', 'sodium free')
    ingredient = free_and_previous_word_to_remove.sub("", ingredient)
    
    # Remove excess spacing in between words after first cleaning pass.
    ingredient = multiple_spaces_to_trim.sub(" ", ingredient)
    
    # Map common plural ingredients to singular ingredients (e.g., "tomatoes" == "tomato").
    for plural, singular in map_plural_to_singular:
        ingredient = re.sub(r"\b{}\b".format(plural), singular, ingredient)
        
    # Remove unuseful words (e.g., "natural", "all-purpose", etc.).
    for phrase in phrases_to_remove:
        ingredient = re.sub(r"\b{}\b".format(phrase), "", ingredient)
    
    # Map several similar phrases to the other equivalents to maintain consistency.
    for pattern, replacement in phrases_to_map:
        if type(pattern) is tuple:
            for val in pattern:
                ingredient = re.sub(r"\b{}\b".format(val), replacement, ingredient)
        elif type(pattern) is str:
            ingredient = re.sub(r"\b{}\b".format(pattern), replacement, ingredient)
    
    # Remove excess spacing in between words after second cleaning pass and leading/trailing whitespace.
    ingredient = multiple_spaces_to_trim.sub(" ", ingredient)
    ingredient = ingredient.strip()
    
    return ingredient


def clean_recipe(recipe):
    """
    Cleans all the ingredients in a single recipe.
    """
    
    # Map the clean function to every ingredient in the recipe.
    recipe['ingredients'] = map(clean_ingredient, recipe['ingredients'])
    
    # Make sure post-cleaning empty strings are removed.
    recipe['ingredients'] = [ingredient for ingredient in recipe['ingredients'] if len(ingredient) > 0]
    
    return recipe


def clean_raw_data(raw_data):
    """
    Cleans all the recipes in the dataset by mapping each recipe to a clean function.
    """
    
    # Map a recipe-cleaning function to every recipe in the data.
    clean_data = np.array(map(clean_recipe, raw_data))
    
    return clean_data



In [11]:
train = clean_raw_data(train)

In [17]:
train['vectorized_ingredients'] = train['ingredients'].map(','.join)


,cuisine,id,ingredients,vectorized_ingredients
0,italian,0,"[penne, shallots, rice vinegar, fresh basil, g...","penne,shallots,rice vinegar,fresh basil,ground..."
1,greek,1,"[sugar, chopped walnuts, filo dough, chopped a...","sugar,chopped walnuts,filo dough,chopped almon..."
2,italian,2,"[fresh rosemary, chopped fresh thyme, fresh or...","fresh rosemary,chopped fresh thyme,fresh orega..."
3,southern_us,3,"[sugar, hot sauce, ramps, vinegar, cream chees...","sugar,hot sauce,ramps,vinegar,cream cheese,gro..."
4,french,4,"[ground cinnamon, panettone, whipped cream, Am...","ground cinnamon,panettone,whipped cream,Amaren..."
5,mexican,5,"[kosher salt, jalapeno chilies, purple onion, ...","kosher salt,jalapeno chilies,purple onion,cano..."
6,southern_us,6,"[water, freshly ground pepper, salt, grits]","water,freshly ground pepper,salt,grits"
7,mexican,7,"[green chile, salt, onions, onion powder, chic...","green chile,salt,onions,onion powder,chicken i..."
8,southern_us,8,"[garlic powder, pork spareribs, sesame seeds, ...","garlic powder,pork spareribs,sesame seeds,crus..."
9,moroccan,9,"[garbanzo beans, fat skimmed chicken broth, gr...","garbanzo beans,fat skimmed chicken broth,groun..."


In [18]:
train['vectorized_ingredients'] = clean_raw_data(train['vectorized_ingredients'])

In [19]:
train

,cuisine,id,ingredients,vectorized_ingredients
0,italian,0,"[penne, shallots, rice vinegar, fresh basil, g...",<map object at 0x0000020D14CFB2B0>
1,greek,1,"[sugar, chopped walnuts, filo dough, chopped a...",<map object at 0x0000020D14CFB2B0>
2,italian,2,"[fresh rosemary, chopped fresh thyme, fresh or...",<map object at 0x0000020D14CFB2B0>
3,southern_us,3,"[sugar, hot sauce, ramps, vinegar, cream chees...",<map object at 0x0000020D14CFB2B0>
4,french,4,"[ground cinnamon, panettone, whipped cream, Am...",<map object at 0x0000020D14CFB2B0>
5,mexican,5,"[kosher salt, jalapeno chilies, purple onion, ...",<map object at 0x0000020D14CFB2B0>
6,southern_us,6,"[water, freshly ground pepper, salt, grits]",<map object at 0x0000020D14CFB2B0>
7,mexican,7,"[green chile, salt, onions, onion powder, chic...",<map object at 0x0000020D14CFB2B0>
8,southern_us,8,"[garlic powder, pork spareribs, sesame seeds, ...",<map object at 0x0000020D14CFB2B0>
9,moroccan,9,"[garbanzo beans, fat skimmed chicken broth, gr...",<map object at 0x0000020D14CFB2B0>


In [21]:
from nltk.stem.lancaster import LancasterStemmer

ModuleNotFoundError: No module named 'nltk'